In [1]:
import pandas as pd

activitats = pd.read_csv('activitats.csv', encoding = 'latin-1')
notes = pd.read_csv('notes.csv', sep=';', encoding = 'latin-1')
notes.rename(columns = {notes.columns[0]: 'userid'}, inplace = True)
trameses = pd.read_csv('trameses.csv', sep=',', encoding = 'latin-1')
trameses.rename(columns = {trameses.columns[0]: 'tramesa_id'}, inplace = True)

In [2]:
notes_aules = notes['aula_id'].unique()
activitats_aules = activitats['aula_id'].unique()
print(notes_aules,activitats_aules)

[ 92 143 176  87 141 184] [ 87  92 114 125 126 141 143 164 165 170 176 184]


Eliminamos los registros de las actividades de las asignatura de las cuales no tenemos notas y aquellos que contengan "PRU", "PROVA" o "OCULT" ya que són pruebas de los docentes.

In [3]:
activitats = activitats[~activitats["aula_id"].isin([114, 125, 126, 164, 165, 170])]
activitats = activitats[~activitats["activitat"].str.contains('PROVA|PRU|OCULT', case=False)]
activitats

,activitat_id,activitat,aula_id,startdate,duedate,grade
0,3,Problema 1.1 - Hello world!!!,87,0,0,100
1,4,Problema 1.2 - Hello world!!! ++,87,0,0,100
2,5,Problema 3a.1: Càlcul edat,87,0,0,100
3,6,Problema 3a.8: Conversió dòlars a euros,87,0,0,100
4,7,Problema 3a.11: Mitjana de notes,87,0,0,100
...,...,...,...,...,...,...
1793,4058,Problema 8.1: Classe Point,184,0,0,100
1794,4059,Problema 8.2: Classe Rectangle,184,0,0,100
1796,4194,(OPCIONAL) Problema 9.24 Triple pitagòric,184,0,0,100
1799,4386,Lliurament Pràctica 5: NEW La Major,176,1571985000,1576745700,100


In [4]:
finals_nan_count = notes.groupby('aula_id').agg(
    total_rows=('F_Grade', 'size'),  # Cuenta el total de filas en cada grupo,
    count_F_NaNs=('F_Grade', lambda x: x.isna().sum())  # Cuenta valores NaN en 'F_Grade'
)

print(finals_nan_count)

         total_rows  count_F_NaNs
aula_id                          
87              112             0
92              252             1
141             113             3
143             238             4
176             248           248
184             125           125


Tenemos 6 asignaturas y podemos eliminar  2 de ellas, ya que aun no se han realizado los exámenes.

In [5]:
notes = notes[~notes["aula_id"].isin([176, 184])]
activitats = activitats[~activitats["aula_id"].isin([176, 184])]

In [6]:
finals_nan_count = notes.groupby('aula_id').agg(
    total_rows=('F_Grade', 'size'),  
    count_F_NaNs=('F_Grade', lambda x: x.isna().sum())  
)

print(finals_nan_count)

         total_rows  count_F_NaNs
aula_id                          
87              112             0
92              252             1
141             113             3
143             238             4


Eliminamos los 8 registros que no tienen nota final

In [7]:
notes = notes[~notes["F_Grade"].isna()]

Contamos el número de actividades por cada asignatura

In [8]:
actxasig = activitats.groupby('aula_id').agg(
    total_rows=('aula_id', 'size')
)

print(actxasig)

         total_rows
aula_id            
87              194
92              141
141             243
143             171


Comprobamos si quedan nans por tratar

In [9]:
nan_counts = activitats.isnull().sum()
nan_counts

activitat_id    0
activitat       0
aula_id         0
startdate       0
duedate         0
grade           0
dtype: int64

In [10]:
nan_counts = notes.isnull().sum()
nan_counts

userid            0
aula_id           0
P_Grade         486
P_Grade_Date    485
F_Grade           0
F_Grade_Date      0
R_Grade         462
R_Grade_Date      0
dtype: int64

In [11]:
finals_nan_count = notes.groupby('aula_id').agg(
    total_rows=('aula_id', 'size'),  
    P_NaNs=('P_Grade', lambda x: x.isna().sum()),
    R_NaNs=('R_Grade', lambda x: x.isna().sum())
)

print(finals_nan_count)

         total_rows  P_NaNs  R_NaNs
aula_id                            
87              112       0      76
92              251     251     170
141             110       1      62
143             234     234     154


Observamos que aula_id 92 y 143 no hacen parciales.

In [12]:
notes = notes[~((notes["aula_id"] == 141) & (notes["P_Grade"].isna()))]

In [13]:
timestamp_min_87 = notes[notes["aula_id"]==87]['F_Grade_Date'].min()
timestamp_max_87 = notes[notes["aula_id"]==87]['F_Grade_Date'].max()
timestamp_min_92 = notes[notes["aula_id"]==92]['F_Grade_Date'].min()
timestamp_max_92 = notes[notes["aula_id"]==92]['F_Grade_Date'].max()
timestamp_min_141 = notes[notes["aula_id"]==141]['F_Grade_Date'].min()
timestamp_max_141 = notes[notes["aula_id"]==141]['F_Grade_Date'].max()
timestamp_min_143 = notes[notes["aula_id"]==143]['F_Grade_Date'].min()
timestamp_max_143 = notes[notes["aula_id"]==143]['F_Grade_Date'].max()

In [14]:
fecha_min_1 = pd.to_datetime(timestamp_min_87, unit='s')
fecha_max_1 = pd.to_datetime(timestamp_max_87, unit='s')
fecha_min_2 = pd.to_datetime(timestamp_min_92, unit='s')
fecha_max_2 = pd.to_datetime(timestamp_max_92, unit='s')
fecha_min_3 = pd.to_datetime(timestamp_min_141, unit='s')
fecha_max_3 = pd.to_datetime(timestamp_max_141, unit='s')
fecha_min_4 = pd.to_datetime(timestamp_min_143, unit='s')
fecha_max_4 = pd.to_datetime(timestamp_max_143, unit='s')


print("Fecha 87:", fecha_min_1, fecha_max_1)
print("Fecha 92:", fecha_min_2, fecha_max_2)
print("Fecha 141:", fecha_min_3, fecha_max_3)
print("Fecha 143:", fecha_min_4, fecha_max_4)

Fecha 87: 2023-01-17 14:00:00 2023-01-19 14:00:00
Fecha 92: 2022-11-07 14:00:00 2022-11-07 14:00:00
Fecha 141: 2024-01-07 14:00:00 2024-01-16 14:00:00
Fecha 143: 2023-11-13 14:00:00 2023-11-13 14:00:00


Se deduce que 87 y 141 son la misma asignatura, así como 92 y 143.


Por lo tanto separamos el dataset en 2 con el objetivo de hacer un modelo para cada asignatura.

In [15]:
notes = notes.drop(columns=["R_Grade", "R_Grade_Date", "P_Grade_Date"], axis=1)
notes

,userid,aula_id,P_Grade,F_Grade,F_Grade_Date
0,176,92,NaN,"9,25",1667829600
1,179,92,NaN,"7,75",1667829600
2,168,92,NaN,"7,75",1667829600
3,401,92,NaN,"3,50",1667829600
4,482,92,NaN,"7,00",1667829600
...,...,...,...,...,...
958,1642,141,"4,75","2,9",1705413600
959,1633,141,"7,6","5,1",1705413600
960,1656,141,"5,88","8,04",1705413600
961,2043,141,"7,8",2,1705413600


In [16]:
notes_A = notes[notes["aula_id"].isin([87, 141])]
notes_B = notes[~notes["aula_id"].isin([87, 141])]
notes_B = notes_B.drop(columns=["P_Grade"], axis=1)

In [17]:
activitats_A = activitats[activitats["aula_id"].isin([87, 141])]
activitats_B = activitats[~activitats["aula_id"].isin([87, 141])]

In [18]:
coinc_A = activitats_A.groupby('activitat').agg(
    total_rows=('activitat', 'size')
)
coinc_A

,total_rows
activitat,
Problema 8.11: Point2D i Point3D,2
Problema 8.14: Baralla de cartes,1
(OPCIONAL) Problema 3b.1: És de l'interval?,2
(OPCIONAL) Problema 3b.20: Iguals o diferents?,2
(OPCIONAL) Problema 3b.33: Zodíac xinès,2
...,...
Recuperació IP - Exercici 3 (1 punt),1
Recuperació IP - Exercici 4 (1 punt),1
Recuperació IP - Exercici 5 (1 punt),1


In [19]:
coinc_B = activitats_B.groupby('activitat').agg(
    total_rows=('activitat', 'size')
)
coinc_B

,total_rows
activitat,
(OPCIONAL) Problema 3b.1: És de l'interval?,2
(OPCIONAL) Problema 3b.20: Iguals o diferents?,2
(OPCIONAL) Problema 3b.33: Zodíac xinès,2
(OPCIONAL) Problema 3b.37: Calculadora,2
(OPCIONAL) Problema 3b.3: Positiu o negatiu?,2
...,...
RECUPERACIÓ Entrega Pràctica 3 - 4: MasterMind,1
RECUPERACIÓ Entrega Pràctica 5: BlackJack,2
RECUPERACIÓ Entrega Pràctica 5: Encriptació,2


Finalmente nos decantamos por hacer un modelo para cada asignatura para simplificar el proceso, ya que aunque sean la misma asignatura, cada año se modifican el numero de entregas.

In [20]:
activitats = activitats.drop(columns=["activitat", "startdate", "duedate"], axis=1)
activitats.rename(columns={'grade': 'gradeMax'}, inplace=True)

In [21]:
activitats_x_aula = {}
notes_x_aula = {}

for aula_id in activitats['aula_id'].unique():
    df_act = activitats[activitats['aula_id'] == aula_id]
    df_notes = notes[notes['aula_id'] == aula_id]
    
    activitats_x_aula[aula_id] = df_act
    notes_x_aula[aula_id] = df_notes


In [22]:
print(activitats_x_aula[87])

     activitat_id  aula_id  gradeMax
0               3       87       100
1               4       87       100
2               5       87       100
3               6       87       100
4               7       87       100
..            ...      ...       ...
207           214       87       100
208           215       87       100
209           216       87       100
499          1652       87       100
859          2221       87       100

[194 rows x 3 columns]


In [23]:
trameses = trameses.drop(columns=["grader", "dategraded"], axis=1)
trameses

,tramesa_id,activitat_id,userid,datesubmitted,grade,nevaluations
0,6,3,31,1658829978,100.0,1
1,7,362,26,1658837049,100.0,0
2,9,362,26,1658837075,100.0,0
3,15,362,31,1662537577,NaN,0
4,16,362,31,1662537593,100.0,0
...,...,...,...,...,...,...
547957,1235631,3567,2644,1729605745,60.0,17
547958,1235635,3573,1828,1729605824,NaN,0
547959,1235643,3573,1828,1729605880,NaN,1
547960,1235646,3567,2650,1729605935,100.0,6


In [24]:
merged_df = pd.merge(trameses, activitats, on='activitat_id')
merged_df['grade'] = merged_df['grade'] / merged_df['gradeMax']
merged_df = merged_df.drop('gradeMax', axis=1)
merged_df

,tramesa_id,activitat_id,userid,datesubmitted,grade,nevaluations,aula_id
0,6,3,31,1658829978,1.0,1,87
1,7,362,26,1658837049,1.0,0,92
2,9,362,26,1658837075,1.0,0,92
3,15,362,31,1662537577,NaN,0,92
4,16,362,31,1662537593,1.0,0,92
...,...,...,...,...,...,...,...
427611,1010148,2466,1673,1722977835,0.8,7,141
427612,1010149,2466,1673,1722977883,1.0,8,141
427613,1010150,2466,1673,1724699940,0.8,9,141
427614,1010151,2466,1673,1724699944,1.0,10,141


In [25]:
idx = merged_df.groupby(['activitat_id', 'userid'])['nevaluations'].idxmax()

merged_df_max_n = merged_df.loc[idx].reset_index(drop=True)

merged_df_max_n

,tramesa_id,activitat_id,userid,datesubmitted,grade,nevaluations,aula_id
0,1222,3,26,1663070055,1.0,9,87
1,6,3,31,1658829978,1.0,1,87
2,1270,3,227,1663074780,1.0,1,87
3,218,3,229,1662996260,1.0,1,87
4,1793,3,241,1663089090,1.0,1,87
...,...,...,...,...,...,...,...
76968,853020,3321,1739,1707207328,NaN,0,141
76969,853019,3321,1746,1707207304,1.0,0,141
76970,853024,3321,1749,1707207356,0.9,0,141
76971,853040,3321,1981,1707207586,NaN,0,141


In [26]:
nan_count = merged_df_max_n.isna().sum()

print("Conteo de NaNs por columna:\n", nan_count)

Conteo de NaNs por columna:
 tramesa_id          0
activitat_id        0
userid              0
datesubmitted       0
grade            5684
nevaluations        0
aula_id             0
dtype: int64


Dado que hemos filtrado por la última entrega de cada user, si existe una entrega con grade NaN se reemplazará por un 0.

In [27]:
merged_df_max_n['grade'] = merged_df_max_n['grade'].fillna(0)


In [33]:
activitats_x_aula[87].columns

Index(['activitat_id', 'aula_id', 'gradeMax'], dtype='object')

Optimizamos el código para crear un DataFrame por cada aula_id, incluyendo las notas parciales (P_Grade), finales (F_Grade), las notas de las entregas, fechas de entrega y número de evaluaciones. Luego, guarda cada DataFrame en un archivo CSV separado.

In [29]:
import pandas as pd

# Obtener usuarios y aulas únicos
unique_users = notes['userid'].unique()
unique_aulas = notes['aula_id'].unique()

# Iterar sobre cada aula_id
for aula_id in unique_aulas:
    users_in_aula = notes[notes['aula_id'] == aula_id]['userid'].unique()
    combinations = pd.MultiIndex.from_product([users_in_aula, [aula_id]], names=['userid', 'aula_id']).to_frame(index=False)
    result_data = []

    # Iterar sobre cada combinación de userid y aula_id
    for _, row in combinations.iterrows():
        userid = row['userid']
        user_aula_trameses = merged_df_max_n[(merged_df_max_n['userid'] == userid) & (merged_df_max_n['aula_id'] == aula_id)]
        user_notes = notes[(notes['userid'] == userid) & (notes['aula_id'] == aula_id)]
        vector = [userid, aula_id]
        vector.append(user_notes['P_Grade'].str.replace(',', '.').astype(float).values[0] if not user_notes['P_Grade'].isna().values[0] else 0.0)
        vector.append(user_notes['F_Grade'].str.replace(',', '.').astype(float).values[0] if not user_notes['F_Grade'].isna().values[0] else 0.0)
        
        # Obtener la fecha del examen final
        final_exam_date = user_notes['F_Grade_Date'].values[0]

        # Iterar sobre cada tramesa_id en el aula_id
        for activitat_id in activitats_x_aula[aula_id]['activitat_id'].unique():
            tramesa = user_aula_trameses[user_aula_trameses['activitat_id'] == activitat_id]
            if not tramesa.empty:
                vector.append(tramesa['grade'].values[0])
                date_submitted = tramesa['datesubmitted'].values[0]
                vector.append(date_submitted if pd.notna(date_submitted) else final_exam_date)
                vector.append(tramesa['nevaluations'].values[0])
            else:
                vector.append(0.0)
                vector.append(final_exam_date)
                vector.append(0)

        result_data.append(vector)

    columns = ['userid', 'aula_id', 'P_Grade', 'F_Grade']
    for activitat_id in activitats_x_aula[aula_id]['activitat_id'].unique():
        columns.extend([f'grade_{activitat_id}', f'date_{activitat_id}', f'nevaluations_{activitat_id}'])

    result_df = pd.DataFrame(result_data, columns=columns)
    result_df.to_csv(f'result_aula_{aula_id}.csv', index=False)
    print(f'DataFrame para aula_id {aula_id} guardado en result_aula_{aula_id}.csv')

DataFrame para aula_id 92 guardado en result_aula_92.csv
DataFrame para aula_id 143 guardado en result_aula_143.csv
DataFrame para aula_id 87 guardado en result_aula_87.csv
DataFrame para aula_id 141 guardado en result_aula_141.csv
